# 04. Data Processing

### Importando as bibliotecas

In [1]:
import sys
sys.path.append('..')
import pandas as pd
from ydata_profiling import ProfileReport
import functions.fn_stats as fn_stats
import functions.fn_charts as fn_charts
import params.consts as consts

### Lendo o dataset tratado e visualizando o overview atual

In [2]:
df = pd.read_csv(consts.DATASET_RAW) # Armazenando o dataset em uma variável

In [ ]:
df # Exibindo as 3 primeiras linhas do dataset

In [ ]:
df.info() # Exibindo as informações das variáveis do dataset

In [ ]:
fn_stats.describe(df) # Usando a função que exibe as estatísticas das colunas numéricas

In [ ]:
df.describe(exclude='number') # Exibindo as estatísticas das colunas categóricas

In [ ]:
df.isnull().sum() # Somando todos os valores nulos de cada coluna

### Corrigindo os tipos de dados das colunas
- **Dt_Customer:** Convertendo essa coluna para o formato de data.

In [8]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%Y-%m-%d') # Convertendo a coluna para o formato de data

### Tratando as colunas com valores nulos/vazios
- **Income:** Como essa coluna possui apenas 24 valores nulos em um univeso de 2.240 (o que representa cerca de 1% da base), optamos por remover essas linhas para seguir o projeto.

In [9]:
df = df.dropna(subset=['Income']) # Removendo as linhas com valores nulos da coluna

### Criando colunas de tempo/idade
- **Dt_Customer => Days_Since_Enrolled:** Criando essa coluna para usá-la de forma mais apropriada para os modelos, onde a informação de tempo/idade é mais relevante que uma data.
- **Dt_Customer => Years_Since_Enrolled:** Criando essa coluna para usá-la de forma mais apropriada para os modelos, onde a informação de tempo/idade é mais relevante que uma data.
- **Year_Birth => Age:** Criando essa coluna para usá-la de forma mais apropriada para os modelos, onde a informação de tempo/idade é mais relevante que uma data.

In [ ]:
df['Days_Since_Enrolled'] = df['Dt_Customer'].max() - df['Dt_Customer'] # Criando a coluna com a diferença do valor máximo das datas e a data de cadastro do cliente
df['Days_Since_Enrolled'] = df['Days_Since_Enrolled'].dt.days # Convertendo o tempo para dias
df['Years_Since_Enrolled'] = df['Days_Since_Enrolled'] // 365 # Criando a coluna apenas com anos inteiros (dividindo com 2 barras)
df['Age'] = df['Dt_Customer'].max().year - df['Year_Birth'] # Criando a coluna com a idade dos clientes, fixando o valor máximmo das datas da campanha para que as idades não fiquem variando de acordo com o tempo que o código é rodado

### Mesclando colunas com muitas categorias
- **AgeGroup:** Criando essa coluna para agrupar os dados em intervalos. 
- **Marital_Status:** Agrupando melhor os dados dessa coluna.

In [ ]:
df['AgeGroup'] = pd.cut( # Criando essa coluna para agrupar as idades em intervalos padrões de cerca de 15 anos
    df['Age'], # Definindo a coluna que será usada para criar os intervalos
    bins=[18, 30, 45, 60, df['Age'].max()], # Definindo os intervalos
    labels=['18-30', '31-45', '46-60', '61+'], # Definindo os rótulos de como os dados vão aparecer no dataset
    include_lowest=True # Incluindo o valor inicial no intervalo
)
df['Marital_Status'] = df['Marital_Status'].replace( # Agrupando melhor os dados dessa coluna
    {
        'Alone': 'Single',
        'Absurd': 'Single',
        'YOLO': 'Single',
        'Together': 'Partner',
        'Married': 'Partner',
        'Widow': 'Single',
        'Single': 'Single',
        'Divorced': 'Single',
    }
)

In [ ]:
display(df['AgeGroup'].value_counts(normalize=True).map('{:.1%}'.format).sort_index()) # Verificando o resultado da transformação na coluna em %
display(df['Marital_Status'].value_counts(normalize=True).map('{:.1%}'.format).sort_index()) # Verificando o resultado da transformação na coluna em %

### Unificando colunas semelhantes
- **Children:** Criando uma coluna com o total de dependentes do cliente.
- **MntTotal:** Criando uma coluna com o total gastos em todas as categorias de produtos.
- **AcceptedCmpTotal:** Criando uma coluna com o total de campanhas aceitas pelo cliente.
- **NumTotalPurchases:** Criando uma coluna com o total de compras feitas pelo cliente.

In [ ]:
df['Children'] = df['Kidhome'] + df['Teenhome'] # Criando a coluna com o total de dependentes do cliente
df['MntTotal'] = ( # Criando a coluna com o total gasto em todas as categorias de produtos
    df['MntWines'] + 
    df['MntFruits'] + 
    df['MntMeatProducts'] + 
    df['MntFishProducts'] + 
    df['MntSweetProducts'] + 
    df['MntGoldProds']
)
df['AcceptedCmpTotal'] = ( # Criando a coluna com o total de campanhas aceitas pelo cliente
    df['AcceptedCmp1'] + 
    df['AcceptedCmp2'] + 
    df['AcceptedCmp3'] + 
    df['AcceptedCmp4'] + 
    df['AcceptedCmp5'] 
)
df['NumTotalPurchases'] = ( # Criando a coluna com o total de compras feitas pelo cliente
    df['NumWebPurchases'] + 
    df['NumCatalogPurchases'] + 
    df['NumStorePurchases'] 
)

In [ ]:
display(df['Children'].value_counts(normalize=True).map('{:.1%}'.format).sort_index()) # Verificando o resultado da transformação na coluna em %
display(df['AcceptedCmpTotal'].value_counts(normalize=True).map('{:.1%}'.format).sort_index()) # Verificando o resultado da transformação na coluna em %

### Criando colunas derivadas de outras
- **HasChildren:** Criando essa coluna caso o cliente tenha dependentes ou não.
- **MntRegularProds:** Criando essa coluna com o valor gasto somente em produtos regulares.
- **HasAcceptedCmp:** Criando essa coluna caso o cliente tenha aceito alguma campanha ou não.

In [ ]:
df['HasChildren'] = df['Children'].apply(lambda x: 1 if x > 0 else 0) # Criando a coluna que verifica se o cliente tem dependentes através de uma função anônima (na própria linha de código)
df['MntRegularProds'] = df['MntTotal'] - df['MntGoldProds'] # Criando a coluna com o valor gasto somente em produtos regulares
df['HasAcceptedCmp'] = df['AcceptedCmpTotal'].apply(lambda x: 1 if x > 0 else 0) # Criando a coluna que verifica se o cliente já aceitou uma campanha através de uma função anônima (na própria linha de código)

### Visualizando o dataset antes de tratar outliers e excluir colunas

In [ ]:
df.head(3) # Exibindo as 3 primeiras linhas do dataset

### Tratando as colunas com valores outliers
- **Age:** Como são poucos outliers dentro do universo dos dados, vamos remover as linhas desses outliers.
- **Income:** Como são poucos outliers dentro do universo dos dados, vamos remover as linhas desses outliers.
- **MntTotal:** Como são poucos outliers dentro do universo dos dados e estão muitos próximos do limite superior, vamos manter as linhas desses outliers.

In [ ]:
fn_charts.boxplots(df,  ['Age', 'Income', 'MntTotal']) # Exibindo os gráficos de boxplots antes do tratamento

In [ ]:
fn_stats.inspect_outliers(df, 'Age') # Usando a função para listar os outliers

In [ ]:
fn_stats.inspect_outliers(df, 'Income') # Usando a função para listar os outliers

In [20]:
df = df.drop(fn_stats.inspect_outliers(df,'Age').index) # Removendo os outliers 
df = df.drop(fn_stats.inspect_outliers(df,'Income').index) # Removendo os outliers 

In [ ]:
fn_charts.boxplots(df,  ['Age', 'Income', 'MntTotal']) # Exibindo os gráficos de boxplots antes do tratamento

### Excluindo as colunas que contém valores únicos
- **ID:** Excluindo a coluna ID para deixar o dataset apenas com informações relevantes, já que uma coluna com identificadores únicos para cada cliente não acrescenta em nada nesse estudo. E lembrando que ao remover a coluna de identificação única, é possível que o dataset passe a ter linhas duplicadas.

In [22]:
df = df.drop('ID', axis=1) # Excluindo as colunas com valores únicos

### Excluindo as colunas que contém valores constantes
- **Z_CostContact:** Excluindo a coluna Z_CostContact para deixar o dataset apenas com informações relevantes, já que uma coluna com valores únicos não acrescenta em nada nesse estudo.
- **Z_Revenue:** Excluindo a coluna Z_Revenue para deixar o dataset apenas com informações relevantes, já que uma coluna com valores únicos não acrescenta em nada nesse estudo.

In [23]:
df = df.drop(['Z_CostContact', 'Z_Revenue'], axis=1) # Excluindo as colunas com valores constantes

### Salvando o dataset processado

In [24]:
df.to_csv(consts.DATASET_PROCESSED, index=False) # Salvando o dataset processado

### Excluindo as colunas auxiliares
- **Dt_Customer:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **Year_Birth:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **Kidhome:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **Teenhome:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **MntWines:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **MntFruits:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **MntMeatProducts:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **MntFishProducts:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **MntSweetProducts:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **AcceptedCmp1:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **AcceptedCmp2:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **AcceptedCmp3:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **AcceptedCmp4:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **AcceptedCmp5:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **NumWebPurchases:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **NumCatalogPurchases:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.
- **NumStorePurchases:** Excluindo a coluna para deixar o dataset apenas com informações relevantes.

In [25]:
df = df.drop( # Excluindo as colunas auxiliares
    [ 
        'Dt_Customer', 
        'Year_Birth', 
        'Kidhome',
        'Teenhome',
        'MntWines', 
        'MntFruits', 
        'MntMeatProducts', 
        'MntFishProducts', 
        'MntSweetProducts',
        'AcceptedCmp1',
        'AcceptedCmp2',
        'AcceptedCmp3',
        'AcceptedCmp4',
        'AcceptedCmp5', 
        'NumWebPurchases', 
        'NumCatalogPurchases', 
        'NumStorePurchases', 
    ], 
    axis=1 # Definindo que colunas serão excluídas
) 

### Ordenando as colunas relacionadas

In [26]:
df = df.reindex(columns=[ # Ordenando as colunas em uma ordem que faz mais sentido para análises
        'Education', 
        'Marital_Status', 
        'Children',
        'HasChildren',
        'Age',
        'AgeGroup',
        'Income',
        'Recency',
        'Complain',
        'Days_Since_Enrolled',
        'Years_Since_Enrolled',
        'NumDealsPurchases',
        'NumWebVisitsMonth',
        'NumTotalPurchases',
        'MntRegularProds',
        'MntGoldProds',
        'MntTotal',
        'AcceptedCmpTotal',
        'HasAcceptedCmp',
        'Response',
    ]
)

### Salvando o dataset lean

In [27]:
df.to_csv(consts.DATASET_LEAN, index=False) # Salvando o dataset lean

### Gerando um novo relatório Profile Report

In [ ]:
profile = ProfileReport(df) # Armazenando em uma variável o ProfileReport

profile.to_file(consts.EDA_1) # Salvando o profile report na pasta reports em formato html